## Wstęp do Uczenia Maszynowego 
#### Laboratorium 02

In [ ]:
# Import potrzebnych pakietów.
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.model_selection import train_test_split

### 0. Drzewa decyzyjne - regresja

Na chwilę wrócimy do zajęć lab 01.

Pracowaliśmy ostatnio nad zadaniem regresji i przewidywaniem wynagrodzenia dla zawodników Baseballu.

Stworzyliśmy pierwszy model i ocenialiśmy jego jakość mierząc błąd średniokwadratowy.

In [ ]:
## Kod pobierający danych
Hitters = pd.read_csv("https://raw.githubusercontent.com/kozaka93/2025Z-MachineLearning/refs/heads/main/labs/lab01/Hitters.csv", index_col =[0])
## Ograniczamy zbiór do trzech zmiennych
Hitters_small = Hitters[["Years", "Hits", "Salary"]]
## Usuwamy braki danych
Hitters_small = Hitters_small.dropna()

## Dzielimy zbior danych na y i X oraz na train i test
y = Hitters_small.Salary
X = Hitters_small.drop(["Salary"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=123)

## Budujemy drzewo z 3 liśćmi
Tree3 = tree.DecisionTreeRegressor(max_leaf_nodes=3)
Tree3 = Tree3.fit(X_train, y_train)

In [ ]:
## Rysunek drzewa
tree.plot_tree(Tree3, 
               feature_names=Tree3.feature_names_in_.tolist(),
               filled=True)

### Regiony 

<img src="Five-Region-Examples.png" width="400"/>

###### Źródło:  James, G., Witten, D., Hastie, T., Tibshirani, R., Taylor, J. (2023). An Introduction to Statistical Learning with Applications in Python, Springer Science+Business Media, New York. https://www.statlearning.com/

-----
##### *Zadanie 1*
------
Na podstawie informacji o utworzonym drzewie zdefiniuj regiony $R_1$, $R_2$ i $R_3$.

-----
##### *Zadanie 2*
------
Na bazie przedstawionych poniżej regionów narysuj odpowiadające im drzewo decyzyjne.

<img src="Regions_Task.png" alt="drawing" width="400"/>

#### Jak konstruować regiony w przypadku drzewa dla zadania regresji?
**Cel**: Znaleźć $R_1, R_2, \dots, R_J$, które minimalizują $RSS$.

$$RSS = \sum_{j = 1}^{J} \sum_{i \in R_j}^{}(y_i - \hat{y}_{R_j})^2$$

Niestety nie jesteśmy w stanie rozpatrzeć wszystkich możliwości - zbyt złożone obliczeniowo.

Używamy podejścia *top-down, greedy*:
- *top-down* - zaczynamy od wszystkich obserwacji w jednym regione i następnie w kolejnych krokach rozdzielamy na mniejsze regiony
- *greedy* - na każdym etapie procesu budowania drzewa dokonywany jest najlepszy podział, nie patrzymy w przyszłość

Proces powatrzamy do momentu spełnienia kryterium stopu, np. w liściu nie może być mniej niż 5 obserwacji.



-----
##### *Zadanie 3*
------

Przeprowadź eksperyment pokazujący wartość miary MSE na zbiorze treningowym i testowym w zależności od wartości parametru `max_leaf_nodes` - zakres 2:30.

`max_leaf_nodes` - Grow a tree with max_leaf_nodes in best-first fashion. Best nodes are defined as relative reduction in impurity. If None then unlimited number of leaf nodes (default=None).

Narysuj wykres przedstawiający wyniki eksperymantu. Jaka jest optymalna wartość parametru `max_leaf_nodes`?

In [ ]:
from sklearn.metrics import mean_squared_error
mse_train = []
mse_test = []

...

### 1. Drzewa decyzyjne - klasyfikacja

In [ ]:
pima = pd.read_csv("pima.csv")
# pima = pd.read_csv("link do raw GitHub")

Więcej o danych: https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database

### Kryterium podziału
a) wskaźnik błędu klasyfikacji (*classification error rate*) = udział obserwacji ze zbioru treningowego w danym regionie, które nie należą do przeważającej klasy.
    
$$E = 1 - max_{k}(\hat{p}_{mk}) $$
 
$\hat{p}_{mk}$ - proporcja obserwacji zbioru treningowego w m-tym regionie pochodząca z k-tej klasy
   
b) indeks Giniego   

 $$G = \sum_{k=1}^{K}\hat{p}_{mk}(1-\hat{p}_{mk})$$

Indeks Giniego przyjmuje małe wartości jeżeli $\hat{p}_{mk}$ jest bliski 0 lub 1. Z tego powodu indeks Giniego jest określany jako miara czystości węzła (*node purity*) - mała wartość wskazuje, że węzeł zawiera głównie obserwacje z jednej klasy.

c) entropia
 
 $$D = - \sum_{k=1}^{K}\hat{p}_{mk}log\hat{p}_{mk}$$

 $0 \leq \hat{p}_{mk} \leq 1 \ \rightarrow 0 \leq -\hat{p}_{mk}log\hat{p}_{mk}$

Podobnie jak indeks Giniego przyjmuje małe wartości (bliskie zero) gdy $\hat{p}_{mk}$ jest bliskie 0 lub 1.

-----
##### *Zadanie 4*
------

Przygotuj zbiór danych pima do dalszej pracy. Podziel na `X` i `y`, a następnie na zbiór treningowy i testowy w proporcji 7:3.

-----
##### *Zadanie 5*
------
Zbuduj model drzewa dla danych z Zadania 1.

In [ ]:
from sklearn.tree import DecisionTreeClassifier


#### Macierz pomyłek (ang. confusion matrix)

#### Jak dobry jest nasz model (zadanie klasyfikacji)?

Drzewa Decyzyjne mogą być używane również do klasyfikacji. W przypadku klasyfikacji używamy innych metryk oceny.

**Macierz pomyłek (ang. confusion matrix)**

Macierz pomyłek jest kluczowym narzędziem do oceny modeli klasyfikacyjnych. 


  * **True Positive (TP):** Model przewidział "tak" i była to prawda.
  * **True Negative (TN):** Model przewidział "nie" i była to prawda.
  * **False Positive (FP):** Model przewidział "tak", ale była to pomyłka (fałszywy alarm).
  * **False Negative (FN):** Model przewidział "nie", ale była to pomyłka (przeoczenie).
  
<img src="confusion_matrix.png" width="400"/>


-----
##### *Zadanie 6*
------
Wyznacz macierz pomyłek dla predykcji na danych treningowych.

In [ ]:
from sklearn.metrics import confusion_matrix




Metryki bazujące na macierzy pomyłek:

  - **Dokładność (Accuracy):** Jak duży odsetek wszystkich prognoz był poprawny.
    $$\text{Accuracy} = \frac{TP + TN}{TP + TN + FP + FN}$$

  - **Czułość (Recall) / Pełność:** Jak duży odsetek wszystkich **rzeczywistych pozytywów** został poprawnie zidentyfikowany.
    $$\text{Recall} = \frac{TP}{TP + FN}$$

  - **Precyzja (Precision):** Jak duży odsetek wszystkich **prognoz pozytywnych** był faktycznie poprawny.
    $$\text{Precision} = \frac{TP}{TP + FP}$$

  - **F1-Score:** Średnia harmoniczna Precyzji i Czułości. Jest używany, gdy chcemy równowagi między tymi dwoma metrykami.
    $$\text{F1-Score} = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}$$

-----

-----
##### *Zadanie 7*
------
Dla utworzonego modelu wylicz powyższe metryki.